# Map options with px and folium for Dash

In [1]:
import pandas as pd
import json
import geopandas as gpd
import plotly.express as px
import folium
pd.options.display.max_columns=None

In [2]:
investments = pd.read_csv("../data/mintic_investments_clean.csv")

We remove the departments classified as national and international

In [3]:
investments_1 = investments.copy()
investments_1 = investments[~investments.departame_cod.isin(['0', 'in'])]
investments_1['departame_cod']= investments_1['departame_cod'].astype('int64')
investments_1.head(5) 

<ipython-input-3-f001c735c633>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  investments_1['departame_cod']= investments_1['departame_cod'].astype('int64')


,anio_corte,fecha_corte,departame_nombre,departame_cod,municipio_nombre,municipio_cod,iniciativa,viceministerio,indicador,descripcion_indicador,inversion,beneficiarios,id
0,2020,2020-12-31,bogota,11,bogota_d.c.,11001,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,773577049710,606,202011001
1,2020,2020-12-31,antioquia,5,medellin,5001,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,114887680650,90,20205001
2,2020,2020-12-31,antioquia,5,apartado,5045,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,70209138175,55,20205045
3,2020,2020-12-31,valle_del_cauca,76,cali,76001,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,59996899895,47,202076001
4,2020,2020-12-31,cundinamarca,25,villapinzon,25873,asesorias_en_teletrabajo,transformacion,numero_de_personas_impactadas_con_la_difusion_...,numero_de_personas_impactadas_con_la_difusion_...,47231602045,37,202025873


Map with plotly express

we import the shp file from the dane geoportal

In [4]:
#all four files must be imported: .shp, .shx, .dbf, .prj of DANE Geoportal
mapa_colombia = gpd.read_file("/Users/SebastianGuzman/Desktop/MGN2020_DPTO_POLITICO/MGN_DPTO_POLITICO.shp")

mapa_colombia['DPTO_CCDGO'] = mapa_colombia['DPTO_CCDGO'].astype('int64')
mapa_colombia = mapa_colombia.sort_values('DPTO_CCDGO').reset_index().drop(columns='index')
dpto_nombres = investments_1[['departame_cod','departame_nombre']].drop_duplicates().sort_values('departame_cod').reset_index()['departame_nombre']
mapa_colombia['departame_nombre'] = dpto_nombres

print(type(mapa_colombia))
print(mapa_colombia.columns)
print(type(mapa_colombia.geometry))
print(mapa_colombia[['DPTO_CCDGO', 'departame_nombre']])

<class 'geopandas.geodataframe.GeoDataFrame'>
Index(['DPTO_CCDGO', 'DPTO_CNMBR', 'DPTO_ANO_C', 'DPTO_ACT_A', 'DPTO_NAREA',
       'DPTO_CSMBL', 'DPTO_VGNC', 'SHAPE_AREA', 'SHAPE_LEN', 'geometry',
       'departame_nombre'],
      dtype='object')
<class 'geopandas.geoseries.GeoSeries'>
    DPTO_CCDGO    departame_nombre
0            5           antioquia
1            8           atlantico
2           11              bogota
3           13             bolivar
4           15              boyaca
5           17              caldas
6           18             caqueta
7           19               cauca
8           20               cesar
9           23             cordoba
10          25        cundinamarca
11          27               choco
12          41               huila
13          44          la_guajira
14          47           magdalena
15          50                meta
16          52             narinio
17          54  norte_de_santander
18          63             quindio
19          66

In [5]:
geo_map_col = json.loads(mapa_colombia.to_json())

In [ ]:
dff1 = investments_1[investments_1['anio_corte']==2016].groupby(['departame_nombre', 'anio_corte']).sum().reset_index()

#Here we can see the syntax used for the creation of choropleth maps:

px.choropleth_mapbox(dff1,                         #Data
        locations='departame_nombre',featureidkey="properties.departame_nombre", #Column containing the identifiers used in the GeoJSON file 
        color='inversion',   #Column giving the color intensity of the region
        geojson=geo_map_col,                          #The GeoJSON file
        zoom=3,                                   #Zoom
        mapbox_style="carto-positron",            #Mapbox style, for different maps you need a Mapbox account and a token
        center={"lat": 5, "lon": -72}, #Center
        color_continuous_scale="Viridis",         #Color Scheme
        opacity=0.5,                              #Opacity of the map
        )

Map with folium

In [ ]:
investments_1['inversion_millions'] = investments_1['inversion']/1000000

dff = investments_1[investments_1['anio_corte']==2016].groupby(['departame_nombre', 'anio_corte']).sum().reset_index() #2016 map
#print(dff.head(10))

m = folium.Map(location=[5, -72], zoom_start=5, tiles='CartoDB positron')
folium.Choropleth(
    geo_data=geo_map_col,
    name="choropleth",
    data=dff,
    columns=["departame_nombre", "inversion_millions"],
    key_on="properties.departame_nombre",#"feature.id",
    fill_color="YlGn",
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name="Inversion (millions)",
).add_to(m)

folium.LayerControl().add_to(m)

m